# Inheritance 2

* [Overview](#overview)
* [Inheritance 2](#sec1)
    * [Protected fields](#sub_sec_1)
    * [Superclass construction](#sub_sec_2)
    * [Overriding methods](#sub_sec_3)
    * [Anonymous subclasses](#sub_sec_4)
    * [Abstract classes & fields](#sub_sec_5)
    * [Construction order](#sub_sec_6)
* [References](#refs)

## <a name="overview"></a> Overview

In this section we continue discussing  inheritance of classes in Scala. Speciffically, in this section, we will consider the following topics 

- Protected fields

- Superclass construction

- Anonymous subclasses 

- Abstract classes & fields

- Construction order

## <a name="sec1"></a> Inheritance 2

We start first we protected fields and methods

### <a name="sub_sec_1"></a> Protected fields

As in Java and C++, you can declare a ﬁeld or a method as ```protected``` . Such a member is accessible from any subclass, but not from other locations. Unlike in Java, a protected member is not visible throughout the package to which the class belongs. (If you want this visibility, you can use a package modiﬁer) [1].

In [1]:
class Element{
    
    protected def speak = println("Hi from element")
    
}

defined class Element

---
**Remark**

In addition to ```private[this]```  there is also a ```protected[this]```. The latter restricts access only to the current object, similar to ```private[this]```.

---

### <a name="sub_sec_2"></a> Superclass construction

We already know that a class can have only one primary constructor which is eventually called by every auxiliary constructor. In fact, an auxiliary constructor should either start with a call to a preceding auxiliary constructor or a call to the primary constructor [1].

The implication of the rule set forth above is that an auxiliary constructor can never invoke a superclass constructor directly [1]. In fact, it is only the primary constructor that can call a superclass constructor [1].

However, the primary constructor is interwined with the class definition [1]. Therefore,
a call to the superclass constructor is similarly intertwined. Let's see an example

In [2]:
class Element(elType: String, idx: Int){
    
}

defined class Element

In [3]:
class Tri(idx: Int) extends Element(elType = "Tri", idx = idx){
    
    private var order = -1
    
    // auxiliary constructor
    def this(order: Int, idx: Int ){
        this(idx)
        this.order = order
        
    }
    
}

defined class Tri

The benefit of intertwining the class and the constructor is that we get very concise code [1].

### <a name="sub_sec_3"></a> Overriding fields

A ﬁeld in Scala consists of three things:

- the field itself

- an accessor function

- a mutator or setter, as long as the field is not declared as a ```val```

In Scala, we can override a ```val``` (or a parameterless ```def``` ) with
another ```val``` ﬁeld of the same name. The subclass has a private ﬁeld and a public
getter, and the getter overrides the superclass getter (or method)

In [4]:
class Item(name: String){
    override def toString = getClass.getName + "[name= " + name + "]"
}



defined class Item

In [5]:
class Milk extends Item("Milk")

defined class Milk

In [6]:
val milk = new Milk
println(milk.toString)

ammonite.$sess.cmd4$Helper$Milk[name= Milk]


milk: Milk = ammonite.$sess.cmd4$Helper$Milk[name= Milk]

In [6]:
class Bullet(name: String) extends Item(name){
    override val name = "Flower"
}

cmd6.sc:2: name is already defined as value name
    override val name = "Flower"
                 ^Compilation Failed

: 

In [ ]:
val bullet = new Bullet("Bullet")
println(bullet.toString)

- A ```def``` can only override another ```def``` .
- A ```val``` can only override another ```val``` or a parameterless ```def``` .
- A ```var``` can only override an abstract ```var```

### <a name="sub_sec_4"></a> Anonymous subclasses

We can make an instance of an anonymous subclass if we include a block
with deﬁnitions or overrides, such as

In [6]:
val tri = new Item("Anonymous"){
    def triName = "I am a TRI" 
}

def makeTri(item: Item{def triName: String}){
    println(item.name + "says " + item.triName)
}

cmd6.sc:6: value name is not a member of cmd6.this.cmd3.Item{def triName: String}
    println(item.name + "says " + item.triName)
                 ^Compilation Failed

: 

### <a name="sub_sec_5"></a>Abstract classes & fields

As in Java, we use the ```abstract``` keyword to denote a class that cannot be instantiated, usually because one or more of its methods are not deﬁned; we call this class an abstract class. 

In Scala, however, we do not use the ```abstract``` keyword for an abstract method but simply
omit its body. As in Java, a class with at least one abstract method must be declared abstract .

In addition to abstract methods, a class can also have abstract ﬁelds. An abstract
ﬁeld is simply a ﬁeld without an initial value. For example

In [ ]:
class MyC{
    // No initializer—this is an 
    // abstract ﬁeld with an abstract getter method
    val id: Int
    
    // Another abstract ﬁeld, 
    // with abstract getter and setter methods
    var id2: Double
}

This class deﬁnes the following [1]

- abstract getter methods for the ```id``` and ```id2``` ﬁelds

- an abstract setter for the ```id2``` ﬁeld

---
**Remark**

The generated Java class has no ﬁelds [1].

---

### <a name="sub_sec_6"></a> Construction order and early deﬁnitions

So what is the order of construction when inheritance is involved? In C++ and Java, the base classe(s) are constructre first and this is makes perfect sense. The same thing happens in Scala

In [4]:
class Base{
    println("Construct Base...")
}

class Derived1 extends Base{
    
  println("Construct Derived1...")
}

class Derived2 extends Derived1{
    
   println("Construct Derived2...")
}

val derived = new Derived2

Construct Base...
Construct Derived1...
Construct Derived2...


defined class Base
defined class Derived1
defined class Derived2
derived: Derived2 = ammonite.$sess.cmd3$Helper$Derived2@1a4d279c

The superclass constructor runs before the subclass constructor.

However, there is a subtle point here. When we override a ```val``` in a subclass that  is used in a superclass constructor the behavior is surprising.

In [3]:
class Base{
    val range: Int = 10
}

class Derived extends Base{
    override val range = 2
}

defined class Base
defined class Derived

In [5]:
val der = new Derived
println(der.range)

2


der: Derived = ammonite.$sess.cmd2$Helper$Derived@5c15b16d

Nothing new here given what we saw in the previously. However, consider the following

In [6]:
class Base{
    val range: Int = 10
    val env: Array[Int] = new Array[Int](range)
}

class Derived extends Base{
    override val range = 2
}

defined class Base
defined class Derived

In [7]:
val der = new Derived
println(der.env.length)

0


der: Derived = ammonite.$sess.cmd5$Helper$Derived@274040cf

Here is what happens [1]

1. The ```Derived``` constructor calls the ```Base``` constructor before doing its own
construction.
2. The ```Base``` constructor sets its range ﬁeld to 10, and then calls ```range()``` in order to initialize ```env```.
3. ```range()``` is overridden to yield the (as yet uninitialized) ```range``` ﬁeld of the ```Derived``` class.
4. The ```range()``` method returns 0 . (That is the initial value of all integer ﬁelds when an object is allocated.)
5. ```env``` is set to an array of length 0 .
6. The ```Derived``` constructor continues, setting its range ﬁeld to 2 .

This is unintuitive at first as  ```range``` appears to be  either 10 or 2 , ```env``` has been set to an array of length 0. Conclusively, we should not rely on the value of a ```val``` in the body of a constructor.

---
**Remark**

In Java, you have a similar issue when you call a method in a superclass constructor. The method might be overridden in a subclass, and it might not do what you want it to do. (In fact, that is the root cause of our problem—the expression range calls the getter method.)[1]

---

So how can we fix this? There are several remedies.

- Declare the ```val``` as final . This is safe but not very ﬂexible.
- Declare the ```val``` as ```lazy``` in the superclass. This is safe but a bit inefﬁcient.
- Use the early deﬁnition syntax in the subclass.

#### The early deﬁnition syntax

The early deﬁnition syntax allows us to initialize ```val``` ﬁelds of a subclass before the
superclass is executed [1].  In essence, we place the ```val``` ﬁelds in a block after the ```extends``` keyword, as shown below

In [10]:
class Derived extends{
    override val range = 2
} with Base

defined class Derived

In [11]:
val der = new Derived
println(der.env.length)

2


der: Derived = ammonite.$sess.cmd9$Helper$Derived@40fb9a2

Note that the right-hand side of an early deﬁnition can only refer to previous early
deﬁnitions, not to other ﬁelds or methods of the class.

## <a name="refs"></a> References

1. Cay Horstmann, ```Scala for the Impatient 1st Edition```